In [1]:
import os
import time
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from pathlib import Path
import pennylane as qml
import gc
import subprocess

# -------------------- Check GPU availability --------------------
# def get_gpu_memory():
#     """Return list of GPU memory usage in MiB."""
#     result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'], 
#                             capture_output=True, text=True)
#     memory = [int(x) for x in result.stdout.strip().split('\n')]
#     return memory

# print("GPU memory usage:")
# mem = get_gpu_memory()
# for i, m in enumerate(mem):
#     print(f"  GPU {i}: {m} MiB used")

# -------------------- User selects GPU --------------------
GPU_ID = 4   # <--- CHANGE THIS to a GPU with low memory (e.g., < 500 MiB)
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
print(f"\nUsing GPU {GPU_ID}")

# -------------------- Device --------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Torch Device:", DEVICE)

# -------------------- Cleanup --------------------
torch.cuda.empty_cache()
gc.collect()

# -------------------- Hyperparameters --------------------
n_qubits = 14
n_layers = 12
N_COMPONENTS = 5
BATCH_SIZE = 64                     # reduced physical batch size
ACCUMULATION_STEPS = 2               # effective batch = 64
LAMBDA_ENTROPY = 0.05                # set to 0 to disable entropy
LEARNING_RATE = 1e-3
MAX_EPOCHS = 150
PATIENCE = 15

print(f"Qubits: {n_qubits}, Layers: {n_layers}, Components: {N_COMPONENTS}")
print(f"Physical batch: {BATCH_SIZE}, Accumulation steps: {ACCUMULATION_STEPS}")
print(f"Effective batch size: {BATCH_SIZE * ACCUMULATION_STEPS}")

# -------------------- Quantum Device --------------------
# if DEVICE.type == "cuda":
#     try:
#         dev = qml.device("lightning.gpu", wires=n_qubits, shots=None)
#         print("Using PennyLane device: lightning.gpu")
#     except:
#         dev = qml.device("default.qubit", wires=n_qubits, shots=None)
#         print("Fallback to PennyLane device: default.qubit")
# else:
#     dev = qml.device("default.qubit", wires=n_qubits, shots=None)
#     print("Using PennyLane device: default.qubit")
dev = qml.device("default.qubit", wires=n_qubits, shots=None)
print("Using PennyLane device: default.qubit")


Using GPU 4
Torch Device: cuda
Qubits: 14, Layers: 12, Components: 5
Physical batch: 64, Accumulation steps: 2
Effective batch size: 128
Using PennyLane device: default.qubit


In [2]:
# -------------------- Load Data --------------------
DRIVE_URL = "https://drive.google.com/uc?id=1PS0eB8dx8VMzVvxNUc6wBzsMRkEKJjWI"
df = pd.read_csv(DRIVE_URL)
print("Total rows:", len(df))
print("Total reactions:", df["Reaction"].nunique())

# -------------------- Physics Feature Engineering --------------------
M_p = 938.272088
M_n = 939.565420
epsilon = 1e-30
LN10 = np.log(10.0)

def get_nucleon_mass(Z, A):
    return Z * M_p + (A - Z) * M_n

mass1 = df.apply(lambda r: get_nucleon_mass(r["Z1"], r["A1"]), axis=1).values
mass2 = df.apply(lambda r: get_nucleon_mass(r["Z2"], r["A2"]), axis=1).values
mu_MeVc2 = (mass1 * mass2) / (mass1 + mass2 + 1e-12)
Ecm = df["E c.m."].astype(float).values
v_over_c = np.sqrt(np.clip(2 * Ecm / (mu_MeVc2 + epsilon), 0, np.inf))
e2_hbar_c = 1 / 137.035999
df["eta"] = (df["Z1"] * df["Z2"]) / (e2_hbar_c * (v_over_c + 1e-16))

log10_sigma_exp = np.log10(np.clip(df["σ"], 1e-30, np.inf))
log10_sigma_cal = np.log10(np.clip(df["σ cal"], 1e-30, np.inf))
log10_Ecm = np.log10(np.clip(df["E c.m."], 1e-30, np.inf))
log10_exp_term = (2 * np.pi * df["eta"]) / LN10

df["log10_S_exp"] = log10_sigma_exp + log10_Ecm + log10_exp_term
df["log10_S_cal"] = log10_sigma_cal + log10_Ecm + log10_exp_term
df["delta_log10_S"] = df["log10_S_exp"] - df["log10_S_cal"]

df["N1"] = df["A1"] - df["Z1"]
df["N2"] = df["A2"] - df["Z2"]
df["Z1Z2_over_Ecm"] = (df["Z1"] * df["Z2"]) / (df["E c.m."] + epsilon)

MAGIC = np.array([2, 8, 20, 28, 50, 82, 126])
def magic_dist(arr):
    return np.min(np.abs(arr[:, None] - MAGIC[None, :]), axis=1)

df["magic_dist_Z1"] = magic_dist(df["Z1"].values)
df["magic_dist_N1"] = magic_dist(df["N1"].values)
df["magic_dist_Z2"] = magic_dist(df["Z2"].values)
df["magic_dist_N2"] = magic_dist(df["N2"].values)

print("Feature engineering complete.")

Total rows: 3532
Total reactions: 213
Feature engineering complete.


In [3]:
# -------------------- 29 Features --------------------
features_train = [
    'E c.m.', 'Z1', 'N1', 'A1',
    'Z2', 'N2', 'A2', 'Q ( 2 n )',
    'Z1Z2_over_Ecm',
    'magic_dist_Z1','magic_dist_N1','magic_dist_Z2','magic_dist_N2',
    'Z3','N3','A3','β P','β T','R B','ħ ω',
    'Projectile_Mass_Actual', 'Target_Mass_Actual', 'Compound_Nucleus_Mass_Actual',
    'Compound_Nucleus_Sp','Compound_Nucleus_Sn',
    'Projectile_Binding_Energy','Target_Binding_Energy',
    'Compound_Nucleus_Binding_Energy','Compound_Nucleus_S2n'
]

# -------------------- Load Reaction Split --------------------
BASE_DIR = "mdn_70_10_20_optimized"
train_reacts = pd.read_csv(os.path.join(BASE_DIR, "train_reactions.csv"))["Reaction"].values
val_reacts   = pd.read_csv(os.path.join(BASE_DIR, "val_reactions.csv"))["Reaction"].values
test_reacts  = pd.read_csv(os.path.join(BASE_DIR, "test_reactions.csv"))["Reaction"].values

print("Train reactions:", len(train_reacts))
print("Val reactions:", len(val_reacts))
print("Test reactions:", len(test_reacts))

train_mask = df["Reaction"].isin(train_reacts)
val_mask   = df["Reaction"].isin(val_reacts)
test_mask  = df["Reaction"].isin(test_reacts)

# -------------------- Prepare Arrays --------------------
X_train_full = df.loc[train_mask | val_mask, features_train].values.astype(np.float32)
y_train_full = df.loc[train_mask | val_mask, "delta_log10_S"].values.astype(np.float32).reshape(-1,1)
X_test = df.loc[test_mask, features_train].values.astype(np.float32)
y_test = df.loc[test_mask, "delta_log10_S"].values.astype(np.float32).reshape(-1,1)

# -------------------- Standardize --------------------
scaler = StandardScaler().fit(X_train_full)
X_train_full_s = scaler.transform(X_train_full)
X_test_s = scaler.transform(X_test)

print("Training samples:", X_train_full_s.shape[0])
print("Test samples:", X_test_s.shape[0])

Train reactions: 149
Val reactions: 21
Test reactions: 43
Training samples: 2847
Test samples: 685


In [4]:
# =========================================================
# Quantum Node and QMDN Model (Batched)
# =========================================================

# QNode definition (uses global n_qubits and dev)
@qml.qnode(dev, interface="torch", diff_method="backprop")
def qnode(weights, inputs):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits), rotation="X")
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))   # shape (batch, 2**n_qubits)

class QMDN(nn.Module):
    def __init__(self, in_dim, n_components=N_COMPONENTS, hidden_dim=32):
        super().__init__()
        self.n_components = n_components
        self.encoder = nn.Linear(in_dim, n_qubits)
        self.weight = nn.Parameter(0.01 * torch.randn(n_layers, n_qubits, 3))

        # Quantum output dimension = 2**n_qubits
        self.fc1 = nn.Linear(2**n_qubits, hidden_dim)
        self.fc_pi = nn.Linear(hidden_dim, n_components)
        self.fc_mu = nn.Linear(hidden_dim, n_components)
        self.fc_sigma = nn.Linear(hidden_dim, n_components)

    def forward(self, x):
        x = x.float()
        x_enc = torch.tanh(self.encoder(x))               # (batch, n_qubits)
        probs = qnode(self.weight, x_enc)                  # (batch, 2**n_qubits)

        # --- FIX: cast probs to same dtype as x (float32) ---
        probs = probs.to(x.dtype)

        h = torch.relu(self.fc1(probs))
        pi_logits = self.fc_pi(h)
        mu = self.fc_mu(h)
        sigma_raw = self.fc_sigma(h)

        pi = F.softmax(pi_logits, dim=1)
        sigma = F.softplus(sigma_raw) + 1e-6
        return pi, mu, sigma

In [5]:
# -------------------- Pure MDN NLL --------------------
def mdn_loss(pi, mu, sigma, y):
    y = y.float()
    yexp = y.repeat(1, mu.shape[1])                     # (batch, n_components)
    log_gauss = (
        -0.5 * ((yexp - mu) / sigma) ** 2
        - torch.log(sigma)
        - 0.5 * np.log(2 * np.pi)
    )
    log_mix = torch.logsumexp(torch.log(pi + 1e-12) + log_gauss, dim=1)
    return -log_mix.mean()

# -------------------- Entropy Regularized Loss --------------------
def mdn_loss_entropy(pi, mu, sigma, y, lambda_entropy):
    nll = mdn_loss(pi, mu, sigma, y)
    entropy = -torch.sum(pi * torch.log(pi + 1e-12), dim=1).mean()
    return nll - lambda_entropy * entropy

# -------------------- DataLoader --------------------
def make_loader(X, y, batch=BATCH_SIZE, shuffle=True):
    return DataLoader(
        TensorDataset(torch.tensor(X, dtype=torch.float32),
                      torch.tensor(y, dtype=torch.float32)),
        batch_size=batch,
        shuffle=shuffle,
        num_workers=0,
        pin_memory=True if DEVICE.type == 'cuda' else False
    )

# -------------------- Train/Validation Split --------------------
val_size = int(0.1 * len(X_train_full_s))
indices = np.random.permutation(len(X_train_full_s))
train_idx = indices[val_size:]
val_idx   = indices[:val_size]

train_loader = make_loader(X_train_full_s[train_idx], y_train_full[train_idx])
val_loader   = make_loader(X_train_full_s[val_idx],   y_train_full[val_idx], shuffle=False)
test_loader  = make_loader(X_test_s, y_test, shuffle=False)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))
print("Test batches:", len(test_loader))

Train batches: 41
Val batches: 5
Test batches: 11


In [6]:
# -------------------- Instantiate Model --------------------
model = QMDN(in_dim=X_train_full_s.shape[1]).to(DEVICE)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

# # -------------------- Test Memory with One Batch --------------------
# print("\nTesting one batch...")
# try:
#     xb, yb = next(iter(train_loader))
#     xb, yb = xb.to(DEVICE), yb.to(DEVICE)
#     with torch.no_grad():
#         pi, mu, sigma = model(xb)
#     mem_used = torch.cuda.memory_allocated()/1e9 if DEVICE.type == 'cuda' else 0
#     print(f"✅ Batch size {BATCH_SIZE} fits! Memory allocated: {mem_used:.2f} GB")
#     del xb, yb, pi, mu, sigma
#     torch.cuda.empty_cache()
#     gc.collect()
# except RuntimeError as e:
#     if "out of memory" in str(e):
#         print(f"❌ Batch size {BATCH_SIZE} does NOT fit. Consider reducing BATCH_SIZE or using gradient accumulation.")
#         print("You can set PHYSICAL_BATCH=32 and ACCUM_STEPS=2 to simulate batch 64.")
#     else:
#         raise e

Model parameters: 525739


In [ ]:
# =========================================================
# Training Loop with Per‑Epoch Component Monitoring (SAVED)
# =========================================================

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
best_val_nll = float("inf")
patience_counter = 0

# ---- Tracking lists ----
epoch_pi_means = []      # store average π per epoch
epoch_val_nll = []        # store validation NLL

print("\n🚀 Starting Training (with gradient accumulation & component tracking)\n")

for epoch in range(MAX_EPOCHS):
    model.train()
    train_loss = 0.0
    train_nll = 0.0
    optimizer.zero_grad()   # reset gradients at epoch start

    for batch_idx, (xb, yb) in enumerate(train_loader):
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)

        pi, mu, sigma = model(xb)

        if LAMBDA_ENTROPY > 0:
            loss = mdn_loss_entropy(pi, mu, sigma, yb, LAMBDA_ENTROPY) / ACCUMULATION_STEPS
        else:
            loss = mdn_loss(pi, mu, sigma, yb) / ACCUMULATION_STEPS

        nll = mdn_loss(pi, mu, sigma, yb) / ACCUMULATION_STEPS   # scaled for logging

        loss.backward()

        # Update weights every ACCUMULATION_STEPS batches
        if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item() * xb.size(0) * ACCUMULATION_STEPS   # rescale
        train_nll  += nll.item()  * xb.size(0) * ACCUMULATION_STEPS

    # Handle any remaining batches (if total batches not divisible by ACCUMULATION_STEPS)
    if (len(train_loader) % ACCUMULATION_STEPS) != 0:
        optimizer.step()
        optimizer.zero_grad()

    train_loss /= len(train_loader.dataset)
    train_nll  /= len(train_loader.dataset)

    # ---------- Validation ----------
    model.eval()
    val_loss = 0.0
    val_nll = 0.0
    pi_accum = []   # collect pi for this epoch

    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pi, mu, sigma = model(xb)

            if LAMBDA_ENTROPY > 0:
                vloss = mdn_loss_entropy(pi, mu, sigma, yb, LAMBDA_ENTROPY)
            else:
                vloss = mdn_loss(pi, mu, sigma, yb)

            val_loss += vloss.item() * xb.size(0)
            val_nll  += mdn_loss(pi, mu, sigma, yb).item() * xb.size(0)
            pi_accum.append(pi.cpu())

    val_loss /= len(val_loader.dataset)
    val_nll  /= len(val_loader.dataset)

    # Average π across validation set
    pi_avg = torch.cat(pi_accum, dim=0).mean(dim=0).numpy()
    epoch_pi_means.append(pi_avg)
    epoch_val_nll.append(val_nll)

    print(f"E{epoch:03d} | train L={train_loss:.6f} NLL={train_nll:.6f} | "
          f"val L={val_loss:.6f} NLL={val_nll:.6f} | "
          f"π avg: {np.round(pi_avg,3)} | "
          f"π std: {np.round(pi_avg.std(),3)}")

    # ---------- Early Stopping ----------
    if val_nll < best_val_nll - 1e-4:
        best_val_nll = val_nll
        torch.save(model.state_dict(), "qmdn_14q_12l_best.pth")
        patience_counter = 0
        print(f"  *** New best model (val NLL: {best_val_nll:.6f}) ***")
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        print(f"🛑 Early stopping after epoch {epoch}")
        break

print("\n✅ Training finished. Best val NLL:", best_val_nll)

# ---- SAVE TRACKING DATA FOR LATER ANALYSIS ----
np.save("epoch_pi_means.npy", np.array(epoch_pi_means))
np.save("epoch_val_nll.npy", np.array(epoch_val_nll))
print("📁 Saved per‑epoch component means and validation loss to disk.")


🚀 Starting Training (with gradient accumulation & component tracking)



In [ ]:
# =========================================================
# After Training: Plot Component Evolution
# =========================================================

import matplotlib.pyplot as plt

# Convert tracking lists to arrays
pi_history = np.array(epoch_pi_means)
n_components = pi_history.shape[1]

plt.figure(figsize=(14,5))

# ---- Component evolution plot ----
plt.subplot(1,2,1)
for k in range(n_components):
    plt.plot(pi_history[:, k], label=f'Component {k}', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Average π weight')
plt.title('Component Evolution During Training')
plt.legend()
plt.grid(alpha=0.3)

# ---- Validation loss plot ----
plt.subplot(1,2,2)
plt.plot(epoch_val_nll, 'r-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Validation NLL')
plt.title('Validation Loss')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Optional: Print final component distribution
print("Final epoch average π:", np.round(pi_history[-1], 4))
print("Final epoch component std:", np.round(pi_history[-1].std(), 4))

In [ ]:
# =========================================================
# Load and Plot Saved Training History (Post‑Training)
# =========================================================
import numpy as np
import matplotlib.pyplot as plt

# Load the saved arrays
pi_history = np.load("epoch_pi_means.npy")
val_nll_history = np.load("epoch_val_nll.npy")

print("Loaded pi_history shape:", pi_history.shape)
print("Loaded val_nll_history length:", len(val_nll_history))

n_components = pi_history.shape[1]

# ---- Recreate the plots ----
plt.figure(figsize=(14,5))

# Component evolution
plt.subplot(1,2,1)
for k in range(n_components):
    plt.plot(pi_history[:, k], label=f'Component {k}', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Average π weight')
plt.title('Component Evolution During Training')
plt.legend()
plt.grid(alpha=0.3)

# Validation loss
plt.subplot(1,2,2)
plt.plot(val_nll_history, 'r-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Validation NLL')
plt.title('Validation Loss')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Optional: print final stats
print("Final epoch average π:", np.round(pi_history[-1], 4))
print("Final epoch component std:", np.round(pi_history[-1].std(), 4))

In [ ]:
# -------------------- Load Best Model --------------------
model.load_state_dict(torch.load("qmdn_14q_12l_best.pth", map_location=DEVICE))
model.eval()

# -------------------- Predict Δlog10S --------------------
all_delta_pred = []
with torch.no_grad():
    for xb, _ in test_loader:
        xb = xb.to(DEVICE)
        pi, mu, sigma = model(xb)
        delta_pred = torch.sum(pi * mu, dim=1, keepdim=True)   # mixture mean
        all_delta_pred.append(delta_pred.cpu().numpy())

delta_pred = np.vstack(all_delta_pred).flatten()
delta_true = y_test.flatten()

print("Test Δlog10S RMSE:", np.sqrt(mean_squared_error(delta_true, delta_pred)))
print("Test Δlog10S R²:", r2_score(delta_true, delta_pred))

# -------------------- Reconstruct σ --------------------
test_local = df.loc[test_mask].reset_index(drop=True)
eta_test = test_local["eta"].values
log10_S_cal_test = test_local["log10_S_cal"].values

log10_S_pred = log10_S_cal_test + delta_pred
log10_sigma_pred = (
    log10_S_pred
    - np.log10(np.clip(test_local["E c.m."].values, 1e-30, np.inf))
    - (2*np.pi*eta_test)/np.log(10.0)
)
sigma_pred = 10**log10_sigma_pred
sigma_true = test_local["σ"].values

print("Test logσ R²:", r2_score(np.log10(sigma_true+1e-30), log10_sigma_pred))

In [ ]:
# -------------------- Collect Component Weights for All Samples --------------------
pi_all = []
with torch.no_grad():
    for xb, _ in train_loader:
        xb = xb.to(DEVICE)
        pi, _, _ = model(xb)
        pi_all.append(pi.cpu().numpy())
    for xb, _ in val_loader:
        xb = xb.to(DEVICE)
        pi, _, _ = model(xb)
        pi_all.append(pi.cpu().numpy())
    for xb, _ in test_loader:
        xb = xb.to(DEVICE)
        pi, _, _ = model(xb)
        pi_all.append(pi.cpu().numpy())

pi_all = np.vstack(pi_all)
print("π array shape:", pi_all.shape)
print("Mean π:", pi_all.mean(axis=0))
print("Standard deviation of π:", pi_all.std(axis=0))

# -------------------- Dominant Component --------------------
dominant = np.argmax(pi_all, axis=1)
unique, counts = np.unique(dominant, return_counts=True)
print("\nDominant component counts:")
for u, c in zip(unique, counts):
    print(f"Component {u}: {c} ({c/len(dominant)*100:.1f}%)")

In [ ]:
# Choose a reaction (e.g., the one you used before)
reaction_name = "12 C + 194 Pt"   # change as needed

reaction_mask = df["Reaction"] == reaction_name
reaction_rows = df.loc[reaction_mask].sort_values("E c.m.")
if len(reaction_rows) == 0:
    print("Reaction not found.")
else:
    # Get indices in the full dataset
    full_indices = reaction_rows.index
    # Get the corresponding π values (if you saved them per sample)
    # Here we need to map full_indices to positions in pi_all.
    # This mapping depends on how you constructed pi_all (concatenation order).
    # If you want to be precise, you should extract π during inference for this reaction only.
    # Simpler: run inference again for this reaction.
    x_reaction = scaler.transform(reaction_rows[features_train].values.astype(np.float32))
    x_tensor = torch.tensor(x_reaction, dtype=torch.float32).to(DEVICE)
    with torch.no_grad():
        pi_reaction, _, _ = model(x_tensor)
    pi_reaction = pi_reaction.cpu().numpy()
    E_vals = reaction_rows["E c.m."].values

    plt.figure(figsize=(10,6))
    for k in range(N_COMPONENTS):
        plt.plot(E_vals, pi_reaction[:, k], marker='o', label=f"Regime {k}")
    plt.xlabel("E c.m. (MeV)")
    plt.ylabel("Component weight π")
    plt.title(f"Regime evolution: {reaction_name}")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()